In [0]:
%sql
USE data_science.default

In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_revenue_growth_rate_view_by_j as
select 
    app_package,
    country_group,
    mediasource,
    tag,
    install_day,
    try_divide(revenue_d3, revenue_d1) as r3_r1,
    try_divide(revenue_d7, revenue_d3) as r7_r3,
    try_divide(revenue_d14, revenue_d7) as r14_r7,
    try_divide(revenue_d30, revenue_d14) as r30_r14,
    try_divide(revenue_d60, revenue_d30) as r60_r30,
    try_divide(revenue_d90, revenue_d60) as r90_r60,
    try_divide(revenue_d120, revenue_d90) as r120_r90,
    try_divide(revenue_d135, revenue_d120) as r135_r120,
    try_divide(revenue_d150, revenue_d135) as r150_r135
from lw_20250903_all_avg_view_by_j
;

In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_predict_revenue_growth_rate_view_by_j as
select
    app_package,
    country_group,
    mediasource,
    tag,
    install_day,
    -- 原始数据
    r3_r1, r7_r3, r14_r7, r30_r14, r60_r30, r90_r60, r120_r90, r135_r120, r150_r135,
    -- 预测数据
    lag(r3_r1, 3) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r3_r1,
    lag(r7_r3, 7) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r7_r3,
    lag(r14_r7, 14) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r14_r7,
    lag(r30_r14, 30) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r30_r14,
    lag(r60_r30, 60) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r60_r30,
    lag(r90_r60, 90) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r90_r60,
    lag(r120_r90, 120) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r120_r90,
    lag(r135_r120, 135) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r135_r120,
    lag(r150_r135, 150) over (partition by app_package, country_group, mediasource, tag order by to_date(install_day, 'yyyyMMdd')) as p_r150_r135
from lw_20250903_revenue_growth_rate_view_by_j
order by app_package, country_group, mediasource, tag, to_date(install_day, 'yyyyMMdd')
;

In [0]:
CREATE OR REPLACE TABLE lw_20250903_predict_revenue_growth_rate_table_by_j as
SELECT * from lw_20250903_predict_revenue_growth_rate_view_by_j;

In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_predict_revenue_view_by_j as
WITH predict_data AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        tag,
        substring_index(tag, '_', length(tag) - length(replace(tag, '_', '')) - 0) as tag_modified,
        install_day,
        p_r3_r1,
        p_r7_r3,
        p_r14_r7,
        p_r30_r14,
        p_r60_r30,
        p_r90_r60,
        p_r120_r90,
        p_r135_r120,
        p_r150_r135
    FROM lw_20250903_predict_revenue_growth_rate_view_by_j
),
revenue_data AS (
    SELECT
        app_package,
        country_group,
        mediasource,
        tag,
        install_day,
        cost,
        revenue_d1,
        revenue_d3,
        revenue_d7,
        revenue_d14,
        revenue_d30,
        revenue_d60,
        revenue_d90,
        revenue_d120,
        revenue_d135,
        revenue_d150
    FROM 
        lw_20250903_prepare4Avg_view_by_j
)
SELECT
    r.app_package,
    r.country_group,
    r.mediasource,
    p.tag,
    r.install_day,
    -- 原始数据
    r.cost,
    r.revenue_d1,
    r.revenue_d3,
    r.revenue_d7,
    r.revenue_d14,
    r.revenue_d30,
    r.revenue_d60,
    r.revenue_d90,
    r.revenue_d120,
    r.revenue_d135,
    r.revenue_d150,
    
    -- 基于r1的预测
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) AS r1_p_r3,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) AS r1_p_r7,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) AS r1_p_r14,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) AS r1_p_r30,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) AS r1_p_r60,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) AS r1_p_r90,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r1_p_r120,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r1_p_r135,
    r.revenue_d1 * COALESCE(p.p_r3_r1, 1) * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r1_p_r150,
    
    -- 基于r3的预测
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) AS r3_p_r7,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) AS r3_p_r14,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) AS r3_p_r30,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) AS r3_p_r60,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) AS r3_p_r90,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r3_p_r120,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r3_p_r135,
    r.revenue_d3 * COALESCE(p.p_r7_r3, 1) * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r3_p_r150,
    
    -- 基于r7的预测
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) AS r7_p_r14,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) AS r7_p_r30,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) AS r7_p_r60,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) AS r7_p_r90,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r7_p_r120,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r7_p_r135,
    r.revenue_d7 * COALESCE(p.p_r14_r7, 1) * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r7_p_r150,
    
    -- 基于r14的预测
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) AS r14_p_r30,
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) AS r14_p_r60,
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) AS r14_p_r90,
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r14_p_r120,
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r14_p_r135,
    r.revenue_d14 * COALESCE(p.p_r30_r14, 1) * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r14_p_r150,
    
    -- 基于r30的预测
    r.revenue_d30 * COALESCE(p.p_r60_r30, 1) AS r30_p_r60,
    r.revenue_d30 * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) AS r30_p_r90,
    r.revenue_d30 * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r30_p_r120,
    r.revenue_d30 * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r30_p_r135,
    r.revenue_d30 * COALESCE(p.p_r60_r30, 1) * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r30_p_r150,
    
    -- 基于r60的预测
    r.revenue_d60 * COALESCE(p.p_r90_r60, 1) AS r60_p_r90,
    r.revenue_d60 * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) AS r60_p_r120,
    r.revenue_d60 * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r60_p_r135,
    r.revenue_d60 * COALESCE(p.p_r90_r60, 1) * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r60_p_r150,
    
    -- 基于r90的预测
    r.revenue_d90 * COALESCE(p.p_r120_r90, 1) AS r90_p_r120,
    r.revenue_d90 * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) AS r90_p_r135,
    r.revenue_d90 * COALESCE(p.p_r120_r90, 1) * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r90_p_r150,
    
    -- 基于r120的预测
    r.revenue_d120 * COALESCE(p.p_r135_r120, 1) AS r120_p_r135,
    r.revenue_d120 * COALESCE(p.p_r135_r120, 1) * COALESCE(p.p_r150_r135, 1) AS r120_p_r150,
    
    -- 基于r135的预测
    r.revenue_d135 * COALESCE(p.p_r150_r135, 1) AS r135_p_r150,
    
    -- 基于r150的预测（实际值）
    r.revenue_d150 AS r150_p_r150

FROM revenue_data r
LEFT JOIN predict_data p ON 
    r.app_package = p.app_package
    AND r.country_group = p.country_group
    AND r.mediasource = p.mediasource
    AND r.install_day = p.install_day
    AND r.tag = p.tag_modified
ORDER BY 
    r.app_package, 
    r.country_group, 
    r.mediasource, 
    p.tag, 
    r.install_day;

In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_real_and_predict_revenue_view_by_j as
SELECT
    app_package,
    country_group,
    mediasource,
    tag,
    install_day,
    cost,
    -- 计算日期差值
    datediff(current_date(), to_date(install_day, 'yyyyMMdd')) as days_diff,
    
    -- revenue_d1: 差值>1时用真实值，否则用r1预测（实际上r1总是真实值）
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 1 
        THEN revenue_d1 
        ELSE revenue_d1 
    END AS revenue_d1,
    
    -- revenue_d3: 差值>3时用真实值，否则用r1预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN revenue_d3 
        ELSE r1_p_r3 
    END AS revenue_d3,
    
    -- revenue_d7: 差值>7时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN revenue_d7 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r7 
        ELSE r1_p_r7 
    END AS revenue_d7,
    
    -- revenue_d14: 差值>14时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN revenue_d14 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r14 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r14 
        ELSE r1_p_r14 
    END AS revenue_d14,
    
    -- revenue_d30: 差值>30时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN revenue_d30 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r30 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r30 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r30 
        ELSE r1_p_r30 
    END AS revenue_d30,
    
    -- revenue_d60: 差值>60时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 60 
        THEN revenue_d60 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN r30_p_r60 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r60 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r60 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r60 
        ELSE r1_p_r60 
    END AS revenue_d60,
    
    -- revenue_d90: 差值>90时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 90 
        THEN revenue_d90 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 60 
        THEN r60_p_r90 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN r30_p_r90 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r90 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r90 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r90 
        ELSE r1_p_r90 
    END AS revenue_d90,
    
    -- revenue_d120: 差值>120时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 120 
        THEN revenue_d120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 90 
        THEN r90_p_r120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 60 
        THEN r60_p_r120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN r30_p_r120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r120 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r120 
        ELSE r1_p_r120 
    END AS revenue_d120,
    
    -- revenue_d135: 差值>135时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 135 
        THEN revenue_d135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 120 
        THEN r120_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 90 
        THEN r90_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 60 
        THEN r60_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN r30_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r135 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r135 
        ELSE r1_p_r135 
    END AS revenue_d135,
    
    -- revenue_d150: 差值>150时用真实值，否则用最佳预测
    CASE 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 150 
        THEN revenue_d150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 135 
        THEN r135_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 120 
        THEN r120_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 90 
        THEN r90_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 60 
        THEN r60_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 30 
        THEN r30_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 14 
        THEN r14_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 7 
        THEN r7_p_r150 
        WHEN datediff(current_date(), to_date(install_day, 'yyyyMMdd')) > 3 
        THEN r3_p_r150 
        ELSE r1_p_r150 
    END AS revenue_d150

FROM lw_20250903_predict_revenue_view_by_j
ORDER BY 
    app_package, 
    country_group, 
    mediasource, 
    tag, 
    to_date(install_day, 'yyyyMMdd');

In [0]:
%sql
create or replace table data_science.default.lw_20250903_real_and_predict_revenue_table_by_j as 
select * from data_science.default.lw_20250903_real_and_predict_revenue_view_by_j
;

计算自然量

In [0]:
%sql
CREATE OR REPLACE VIEW lw_20250903_organic_revenue_rate_view_by_j as
SELECT
    app_package,
    country_group,
    tag,
    install_day,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d7 ELSE 0 END), SUM(revenue_d7)) AS organic_revenue_rate_d7,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d14 ELSE 0 END), SUM(revenue_d14)) AS organic_revenue_rate_d14,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d30 ELSE 0 END), SUM(revenue_d30)) AS organic_revenue_rate_d30,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d60 ELSE 0 END), SUM(revenue_d60)) AS organic_revenue_rate_d60,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d90 ELSE 0 END), SUM(revenue_d90)) AS organic_revenue_rate_d90,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d120 ELSE 0 END), SUM(revenue_d120)) AS organic_revenue_rate_d120,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d135 ELSE 0 END), SUM(revenue_d135)) AS organic_revenue_rate_d135,
    try_divide(SUM(CASE WHEN mediasource IN ('organic', 'Organic') THEN revenue_d150 ELSE 0 END), SUM(revenue_d150)) AS organic_revenue_rate_d150
FROM data_science.default.lw_20250903_real_and_predict_revenue_table_by_j
GROUP BY app_package, country_group, tag, install_day
;

In [0]:
select * from lw_20250903_organic_revenue_rate_view_by_j
where app_package = 'com.fun.lastwar.gp'
and country_group = 'US'
and tag = 'aos_gpir_cohort_onlyprofit_raw_avg28'
and install_day between '20250501' and '20250601'
order by install_day desc;